# 📊 Amazon Sales Analysis

This notebook shows step-by-step cleaning and analysis of the Amazon Sales dataset.
- Data Cleaning
- Exploratory Data Analysis (EDA)
- Exporting Cleaned Dataset
- Generating Charts for Insights
- Exporting Summary CSVs (KPI, Monthly, SKU, State, Category)


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Load dataset
df = pd.read_csv('Amazon Sale Report.csv')
df.head()

## 1. Dataset Overview

In [ ]:
print("Shape:", df.shape)
print("\nData Types:\n", df.dtypes)
print("\nMissing Values:\n", df.isnull().sum())
print("\nDuplicates:", df.duplicated().sum())

## 2. Data Cleaning

In [ ]:
# Drop duplicates
df.drop_duplicates(inplace=True)

# Convert order_date
if 'order_date' in df.columns:
    df['order_date'] = pd.to_datetime(df['order_date'], errors='coerce')

# Fill missing values
if 'ship-state' in df.columns:
    df['ship-state'].fillna('Unknown', inplace=True)

# Calculate revenue if not exists
if 'revenue' not in df.columns:
    if 'amount' in df.columns and 'qty' in df.columns:
        df['revenue'] = df['amount'] * df['qty']

df.info()

## 3. Export Cleaned Dataset

In [ ]:
Path('outputs').mkdir(exist_ok=True)
df.to_csv('outputs/amazon_sales_clean.csv', index=False)
print("✅ Cleaned dataset exported!")

## 4. Exploratory Data Analysis (EDA)

In [ ]:
# Daily Revenue Trend
if 'order_date' in df.columns:
    daily = df.groupby('order_date')['revenue'].sum().reset_index()
    plt.figure(figsize=(12,6))
    sns.lineplot(x='order_date', y='revenue', data=daily)
    plt.title('Daily Revenue Trend')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig('outputs/daily_revenue.png')
    plt.show()

In [ ]:
# Monthly Revenue Trend
if 'order_date' in df.columns:
    df['month'] = df['order_date'].dt.to_period('M')
    monthly = df.groupby('month')['revenue'].sum().reset_index()
    plt.figure(figsize=(12,6))
    sns.barplot(x='month', y='revenue', data=monthly, color='purple')
    plt.title('Monthly Revenue Trend')
    plt.xticks(rotation=90)
    plt.tight_layout()
    plt.savefig('outputs/monthly_revenue.png')
    plt.show()

In [ ]:
# Top 10 SKUs
if 'sku' in df.columns:
    top_skus = df.groupby('sku')['revenue'].sum().reset_index().sort_values('revenue', ascending=False).head(10)
    plt.figure(figsize=(10,6))
    sns.barplot(y='sku', x='revenue', data=top_skus, palette='Blues_r')
    plt.title('Top 10 SKUs by Revenue')
    plt.tight_layout()
    plt.savefig('outputs/top_skus.png')
    plt.show()

In [ ]:
# Top Categories
if 'category' in df.columns:
    top_cats = df.groupby('category')['revenue'].sum().reset_index().sort_values('revenue', ascending=False).head(10)
    plt.figure(figsize=(10,6))
    sns.barplot(y='category', x='revenue', data=top_cats, palette='Greens_r')
    plt.title('Top Categories by Revenue')
    plt.tight_layout()
    plt.savefig('outputs/top_categories.png')
    plt.show()

In [ ]:
# Top States
if 'ship-state' in df.columns:
    top_states = df.groupby('ship-state')['revenue'].sum().reset_index().sort_values('revenue', ascending=False).head(10)
    plt.figure(figsize=(10,6))
    sns.barplot(y='ship-state', x='revenue', data=top_states, palette='Reds_r')
    plt.title('Top 10 States by Revenue')
    plt.tight_layout()
    plt.savefig('outputs/top_states.png')
    plt.show()

## 5. Export Summary CSVs

In [ ]:
# 1. KPI Summary
kpi = {
    "Total Revenue": [df['revenue'].sum()],
    "Total Orders": [df['order id'].nunique() if 'order id' in df.columns else len(df)],
    "Average Order Value": [df['revenue'].mean()]
}
pd.DataFrame(kpi).to_csv('outputs/kpi_summary.csv', index=False)

# 2. Monthly Sales Summary
if 'month' in df.columns:
    monthly.to_csv('outputs/monthly_sales_summary.csv', index=False)

# 3. SKU Summary
if 'sku' in df.columns:
    sku_summary = df.groupby('sku').agg({"revenue":"sum", "qty":"sum"}).reset_index()
    sku_summary.to_csv('outputs/sku_summary.csv', index=False)

# 4. State Summary
if 'ship-state' in df.columns:
    state_summary = df.groupby('ship-state')['revenue'].sum().reset_index()
    state_summary.to_csv('outputs/state_summary.csv', index=False)

# 5. Category Summary
if 'category' in df.columns:
    cat_summary = df.groupby('category')['revenue'].sum().reset_index()
    cat_summary.to_csv('outputs/category_summary.csv', index=False)

print("✅ All summary CSVs exported successfully!")

## 6. Insights Summary

In [ ]:
print("✅ Dataset cleaned and visuals generated.")
print("🔹 Total Revenue:", df['revenue'].sum())
if 'order id' in df.columns:
    print("🔹 Total Orders:", df['order id'].nunique())
print("🔹 Average Order Value:", df['revenue'].mean())